# DeepAgent 기반의 Research Agent

## 주요 특징

1. **Multi-SubAgent 시스템**: 세 가지 전문화된 SubAgent (researcher, explorer, synthesizer)
2. **CompiledSubAgent 패턴**: Researcher는 자율적 DeepAgent로 "넓게 탐색 → 깊게 파기" 패턴 수행
3. **Skills System**: Progressive Disclosure를 통한 스킬 관리
4. **Filesystem Backend**: 연구 결과의 영구 저장

## 아키텍처 개요

```
Main Orchestrator (create_deep_agent)
    │
    ├── researcher (CompiledSubAgent - 자율적 DeepAgent)
    │       └── 자체 계획 루프 + 반성 + 컨텍스트 관리
    │
    ├── explorer (Simple SubAgent)
    │       └── 빠른 읽기 전용 탐색
    │
    └── synthesizer (Simple SubAgent)
            └── 연구 결과 통합
```

In [1]:
import sys
from pathlib import Path

from dotenv import load_dotenv

load_dotenv(".env", override=True)

# 프로젝트 루트를 Python 경로에 추가 (노트북에서 모듈 import를 위해)
PROJECT_ROOT = Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

In [2]:
# 연구 도구 살펴보기
from research_agent.tools import tavily_search, think_tool

print(f"1. tavily_search: {tavily_search.description}")
print(f"2. think_tool: {think_tool.description}")

1. tavily_search: 주어진 쿼리로 웹을 검색한다.

Tavily를 사용해 관련 URL을 찾고, 전체 웹페이지 콘텐츠를 마크다운으로 가져와 반환한다.

Args:
    query: 실행할 검색 쿼리
    max_results: 반환할 최대 결과 수 (기본값: 1)
    topic: 주제 필터 - 'general', 'news', 또는 'finance' (기본값: 'general')

Returns:
    전체 웹페이지 콘텐츠가 포함된 포맷팅된 검색 결과
2. think_tool: 연구 진행 상황과 의사결정을 위한 전략적 성찰 도구.

각 검색 후 결과를 분석하고 다음 단계를 체계적으로 계획하기 위해 이 도구를 사용한다.
이는 품질 높은 의사결정을 위해 연구 워크플로우에 의도적인 멈춤을 만든다.

사용 시점:
- 검색 결과를 받은 후: 어떤 핵심 정보를 찾았는가?
- 다음 단계를 결정하기 전: 포괄적으로 답변할 수 있을 만큼 충분한가?
- 연구 공백을 평가할 때: 아직 누락된 구체적인 정보는 무엇인가?
- 연구를 마무리하기 전: 지금 완전한 답변을 제공할 수 있는가?

성찰에 포함해야 할 내용:
1. 현재 발견의 분석 - 어떤 구체적인 정보를 수집했는가?
2. 공백 평가 - 어떤 중요한 정보가 아직 누락되어 있는가?
3. 품질 평가 - 좋은 답변을 위한 충분한 증거/예시가 있는가?
4. 전략적 결정 - 검색을 계속해야 하는가, 답변을 제공해야 하는가?

Args:
    reflection: 연구 진행 상황, 발견, 공백, 다음 단계에 대한 상세한 성찰

Returns:
    의사결정을 위해 성찰이 기록되었다는 확인


---

## 자율적 연구 에이전트 (Autonomous Researcher)

### 핵심 개념: CompiledSubAgent vs Simple SubAgent

| 유형 | 구조 | 특징 |
|------|------|------|
| **Simple SubAgent** | `{name, description, system_prompt, tools}` | 단일 응답, 미들웨어에서 컴파일 |
| **CompiledSubAgent** | `{name, description, runnable}` | 자체 DeepAgent, 다중 턴 실행 |

### 자율적 연구 워크플로우: "넓게 탐색 → 깊게 파기"

```
┌─────────────────────────────────────────────────────────────────┐
│                    AUTONOMOUS RESEARCH LOOP                      │
├─────────────────────────────────────────────────────────────────┤
│  Phase 1: Exploratory Search (1-2회)                             │
│      └── 광범위한 검색으로 연구 환경 파악                          │
│      └── think_tool로 유망한 방향 2-3개 식별                       │
│                                                                   │
│  Phase 2: Directed Research (방향당 1-2회)                        │
│      └── 각 방향에 대한 집중 검색                                  │
│      └── think_tool로 인사이트 평가                                │
│                                                                   │
│  Phase 3: Synthesis                                               │
│      └── 모든 발견을 구조화된 형태로 통합                          │
└─────────────────────────────────────────────────────────────────┘
```

### 에이전트 연구 기법

[Anthropic 에서 제시한 프롬프팅 기법](https://youtu.be/XSZP9GhhuAc?si=zowpViL-2j-vI9hA)을 활용합니다:

1. **에이전트처럼 생각하기**: 질문을 꼼꼼히 읽고, 넓게 시작해서 좁혀가기
2. **하드 리밋 설정**: 검색 도구 2-5회, 최대 5-6회로 제한
3. **생각의 과정 드러내기**: 각 검색 후 `think_tool`로 결과 분석

In [3]:
# 프롬프트 및 유틸리티 임포트
from research_agent.prompts import (
    EXPLORER_INSTRUCTIONS,
    RESEARCH_WORKFLOW_INSTRUCTIONS,
    SUBAGENT_DELEGATION_INSTRUCTIONS,
    SYNTHESIZER_INSTRUCTIONS,
)
from research_agent.researcher.prompts import AUTONOMOUS_RESEARCHER_INSTRUCTIONS
from research_agent.utils import format_messages, show_prompt

In [4]:
show_prompt(AUTONOMOUS_RESEARCHER_INSTRUCTIONS[:2000] + "\n\n... (truncated)")

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  You are an autonomous research agent. Your job is to thoroughly research a topic by following a                │
│  "breadth-first, then depth" approach.                                                                          │
│                                                                                                                 │
│  For context, today's date is {date}.                                                                           │
│                                                                                                                 │
│  ## Your Capabilities                                                                                           │
│                                                                                                                 │
│  You have access to:                                                                                            │
│  - **tavily_search**: Web search with full content extraction                                                   │
│  - **think_tool**: Reflection and strategic planning                                                            │
│  - **write_todos**: Self-planning and progress tracking                                                         │
│                                                                                                                 │
│  ## Autonomous Research Workflow                                                                                │
│                                                                                                                 │
│  ### Phase 1: Exploratory Search (1-2 searches)                                                                 │
│                                                                                                                 │
│  **Goal**: Get the lay of the land                                                                              │
│                                                                                                                 │
│  Start with broad searches to understand:                                                                       │
│  - Key concepts and terminology in the field                                                                    │
│  - Major players, sources, and authorities                                                                      │
│  - Recent trends and developments                                                                               │
│  - Potential sub-topics worth exploring                                                                         │
│                                                                                                                 │
│  After each search, **ALWAYS** use think_tool to:                                                               │
│  ```                                                                                                            │
│  "What did I learn? Key concepts are: ...                                                                       │
│  What are 2-3 promising directions for deeper research?                                                         │
│  1. Direction A: [reason]                                                                                       │
│  2. Direction B: [reason]                                                                                       │
│  3. Direction C: [reason]                                                                                       │
│  Do I need more exploration, or can I proceed to Phase 2?"                                                      │
│  ```                                                  

## SubAgent 구조 비교

아래에서 Simple SubAgent와 CompiledSubAgent의 차이를 살펴봅니다.

In [5]:
# SubAgent 구조 비교
from research_agent.agent import ALL_SUBAGENTS, SIMPLE_SUBAGENTS
from research_agent.researcher import get_researcher_subagent

print("=== SubAgent 구조 비교 ===\n")

# 1. Researcher (CompiledSubAgent)
researcher = ALL_SUBAGENTS[0]
print("1. Researcher (CompiledSubAgent)")
print(f"   - name: {researcher['name']}")
print(f"   - has 'runnable': {'runnable' in researcher}")
print(f"   - runnable type: {type(researcher.get('runnable')).__name__}")
print(f"   - description: {researcher['description'][:60]}...")
print()

# 2. Explorer (Simple SubAgent)
explorer = ALL_SUBAGENTS[1]
print("2. Explorer (Simple SubAgent)")
print(f"   - name: {explorer['name']}")
print(f"   - has 'system_prompt': {'system_prompt' in explorer}")
print(f"   - has 'runnable': {'runnable' in explorer}")
print()

# 3. Synthesizer (Simple SubAgent)
synthesizer = ALL_SUBAGENTS[2]
print("3. Synthesizer (Simple SubAgent)")
print(f"   - name: {synthesizer['name']}")
print(f"   - has 'system_prompt': {'system_prompt' in synthesizer}")
print(f"   - has 'runnable': {'runnable' in synthesizer}")

=== SubAgent 구조 비교 ===

1. Researcher (CompiledSubAgent)
   - name: researcher
   - has 'runnable': True
   - runnable type: CompiledStateGraph
   - description: Autonomous deep research agent with self-planning and 'bread...

2. Explorer (Simple SubAgent)
   - name: explorer
   - has 'system_prompt': True
   - has 'runnable': False

3. Synthesizer (Simple SubAgent)
   - name: synthesizer
   - has 'system_prompt': True
   - has 'runnable': False


---

## Research Orchestrator 에이전트

`research_agent.agent` 모듈에서 이미 구성된 에이전트를 가져옵니다.

### 구성 요소:
- **모델**: `gpt-4.1` (temperature=0.0)
- **백엔드**: CompositeBackend (FileSystem + State)
- **SubAgents**: researcher (CompiledSubAgent), explorer, synthesizer
- **미들웨어**: SkillsMiddleware (Progressive Disclosure)

In [6]:
# research_agent 모듈에서 구성된 에이전트 가져오기
from research_agent.agent import ALL_SUBAGENTS, agent, model

print("=== Research Agent 구성 ===")
print(f"Model: {model.model_name}")
print(f"SubAgents: {[s['name'] for s in ALL_SUBAGENTS]}")
print(f"Agent type: {type(agent).__name__}")

=== Research Agent 구성 ===
Model: gpt-4.1
SubAgents: ['researcher', 'explorer', 'synthesizer']
Agent type: CompiledStateGraph


---

## 연구 실행

에이전트에게 연구 요청을 보내고 결과를 확인합니다.

자율적 연구 에이전트는:
1. 오케스트레이터가 `task(researcher)` 호출
2. Researcher가 자체적으로 Phase 1-2-3 수행
3. 구조화된 결과를 오케스트레이터에 반환

In [7]:
# 연구 요청 실행
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "Research context engineering approaches used to build AI agents. Speak in Korean.",
            }
        ],
    },
)

print(f"Messages: {len(result['messages'])} 개")
print(f"Files: {list(result.get('files', {}).keys())}")

Messages: 28 개
Files: []


### 메시지 흐름 확인

에이전트 실행 과정의 메시지를 시각화합니다.

In [8]:
format_messages(result["messages"])

╭───────────────────────────────────────────────────── Human ─────────────────────────────────────────────────────╮
│ Research context engineering approaches used to build AI agents. Speak in Korean.                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────────── AI ───────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ Tool Call: write_todos                                                                                          │
│    Args: {                                                                                                      │
│   "todos": [                                                                                                    │
│     {                                                                                                           │
│       "content": "사용자 요청을 /research_request.md에 저장",                                                   │
│       "status": "in_progress"                                                                                   │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "AI 에이전트 구축에 사용되는 컨텍스트 엔지니어링 접근법에 대해 연구할 구체적인 질문을          │
│ 도출하고, 연구 계획을 수립",                                                                                    │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "연구 계획에 따라 연구를 하위 에이전트에 위임하여 자료를 수집",                                │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "수집된 자료를 종합 및 인용 정리",                                                             │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "최종 보고서를 /final_report.md에 작성 (한국어, 인용 포함)",                                   │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "/research_request.md와 비교하여 모든 요구사항이 충족되었는지 검증",                           │
│       "status": "pending"                                                                                       │
│     }                                                                                                           │
│   ]                                                                                                             │
│ }                                                                                                               │
│    ID: call_6XvIJLkJZHBkfZXpcXsvf7dV                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ Updated todo list to [{'content': '사용자 요청을 /research_request.md에 저장', 'status': 'in_progress'},        │
│ {'content': 'AI 에이전트 구축에 사용되는 컨텍스트 엔지니어링 접근법에 대해 연구할 구체적인 질문을 도출하고,     │
│ 연구 계획을 수립', 'status': 'pending'}, {'content': '연구 계획에 따라 연구를 하위 에이전트에 위임하여 자료를   │
│ 수집', 'status': 'pending'}, {'content': '수집된 자료를 종합 및 인용 정리', 'status': 'pending'}, {'content':   │
│ '최종 보고서를 /final_report.md에 작성 (한국어, 인용 포함)', 'status': 'pending'}, {'content':                  │
│ '/research_request.md와 비교하여 모든 요구사항이 충족되었는지 검증', 'status': 'pending'}]                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────────── AI ───────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ Tool Call: write_file                                                                                           │
│    Args: {                                                                                                      │
│   "file_path": "/research_request.md",                                                                          │
│   "content": "AI 에이전트 구축에 사용되는 컨텍스트 엔지니어링 접근법을 연구해 주세요. (한국어로 작성)"          │
│ }                                                                                                               │
│    ID: call_Q4zD3BuMuFPlFpE8aMh3OI07                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ Cannot write to /research_request.md because it already exists. Read and then make an edit, or write to a new   │
│ path.                                                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────────── AI ───────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ Tool Call: read_file                                                                                            │
│    Args: {                                                                                                      │
│   "file_path": "/research_request.md",                                                                          │
│   "offset": 0,                                                                                                  │
│   "limit": 100                                                                                                  │
│ }                                                                                                               │
│    ID: call_blmgCqeo812n68sY7m1ascbh                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│      1  연구 주제: AI 에이전트 구축에 사용되는 컨텍스트 엔지니어링 접근법에 대해 조사하라.                      │
│      2                                                                                                          │
│      3  세부 요구사항:                                                                                          │
│      4  - Context Engineering이란 무엇인지                                                                      │
│      5  - 연구 에이전트형 AI 시스템에서의 Context Engineering 적용 사례                                         │
│      6  - 파일시스템 기반 컨텍스트 관리 시스템에서의 Context Engineering 적용 사례                              │
│      7  - 최신 연구 동향 및 실제 적용 예시                                                                      │
│      8  - 참고문헌 및 인용 포함                                                                                 │
│      9  - 결과물은 한국어로 작성할 것                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────────── AI ───────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ Tool Call: write_todos                                                                                          │
│    Args: {                                                                                                      │
│   "todos": [                                                                                                    │
│     {                                                                                                           │
│       "content": "사용자 요청을 /research_request.md에 저장",                                                   │
│       "status": "completed"                                                                                     │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "AI 에이전트 구축에 사용되는 컨텍스트 엔지니어링 접근법에 대해 연구할 구체적인 질문을          │
│ 도출하고, 연구 계획을 수립",                                                                                    │
│       "status": "completed"                                                                                     │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "연구 계획에 따라 연구를 하위 에이전트에 위임하여 자료를 수집",                                │
│       "status": "completed"                                                                                     │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "수집된 자료를 종합 및 인용 정리",                                                             │
│       "status": "completed"                                                                                     │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "최종 보고서를 /final_report.md에 작성 (한국어, 인용 포함)",                                   │
│       "status": "completed"                                                                                     │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "/research_request.md와 비교하여 모든 요구사항이 충족되었는지 검증",                           │
│       "status": "completed"                                                                                     │
│     }                                                                                                           │
│   ]                                                                                                             │
│ }                                                                                                               │
│    ID: call_LXLb7OSu9fyLYBC6bORfZzsk                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│ Updated todo list to [{'content': '사용자 요청을 /research_request.md에 저장', 'status': 'completed'},          │
│ {'content': 'AI 에이전트 구축에 사용되는 컨텍스트 엔지니어링 접근법에 대해 연구할 구체적인 질문을 도출하고,     │
│ 연구 계획을 수립', 'status': 'completed'}, {'content': '연구 계획에 따라 연구를 하위 에이전트에 위임하여 자료를 │
│ 수집', 'status': 'completed'}, {'content': '수집된 자료를 종합 및 인용 정리', 'status': 'completed'},           │
│ {'content': '최종 보고서를 /final_report.md에 작성 (한국어, 인용 포함)', 'status': 'completed'}, {'content':    │
│ '/research_request.md와 비교하여 모든 요구사항이 충족되었는지 검증', 'status': 'completed'}]                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────────── AI ───────────────────────────────────────────────────────╮
│ 요청하신 "AI 에이전트 구축에 사용되는 컨텍스트 엔지니어링 접근법"에 대한 최신 연구와 실제 사례를 종합하여,      │
│ 아래와 같이 보고서를 작성하였습니다.                                                                            │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ # AI 에이전트 구축을 위한 컨텍스트 엔지니어링 접근법 연구                                                       │
│                                                                                                                 │
│ ## 1. Context Engineering이란 무엇이며, 주요 개념과 원리는 무엇인가?                                            │
│                                                                                                                 │
│ 컨텍스트 엔지니어링(Context Engineering)은 AI 에이전트, 특히 멀티에이전트 시스템에서 각 에이전트가 최적의       │
│ 의사결정을 내릴 수 있도록 필요한 정보(컨텍스트)를 설계, 관리, 전달하는 일련의 전략과 기술을 의미합니다. 이는    │
│ 단순히 프롬프트를 설계하는 수준을 넘어, 시스템 전체의 집단적 지식, 이전 결정, 도구 피드백, 외부 데이터 등       │
│ 다양한 맥락 정보를 동적으로 조합·제공함으로써, 에이전트 간 협업의 정확성과 일관성을 극대화하는 것이 핵심        │
│ 원리입니다[1][4].                                                                                               │
│                                                                                                                 │
│ 컨텍스트의 유형은 크게 ① 명령(프롬프트, 규칙, 예시), ② 지식(도메인 정보, 검색 데이터, 메모리), ③ 도구           │
│ 피드백(API 결과, 런타임 신호)로 구분됩니다. 효과적인 컨텍스트 엔지니어링을 위해서는 정보의 저장(외부 메모리,    │
│ 파일시스템), 선택(RAG, 유사도 검색), 압축(요약, 필터링), 격리(역할별 컨텍스트 분리) 등 다양한 전략이            │
│ 활용됩니다[4].                                                                                                  │
│                                                                                                                 │
│ ## 2. 연구 에이전트형 AI 시스템(예: LLM 기반 연구 에이전트)에서의 Context Engineering 적용 사례와 구체적 방법론 │
│                                                                                                                 │
│ 연구 에이전트형 AI 시스템에서는 복잡한 연구 과제를 해결하기 위해 여러 특화 에이전트가 협업하는 멀티에이전트     │
│ 구조가 주로 채택됩니다. 대표적으로 Anthropic의 멀티에이전트 연구 시스템은 Opus 4 리드 에이전트가 Sonnet 4       │
│ 서브에이전트들을 조율하여, 각 서브에이전트가 실시간 데이터 수집, 통계 분석, 전문가 의견 요약 등 역할을          │
│ 분담하고, 리드 에이전트가 결과를 통합·검증하는 방식으로 동작합니다. 이 시스템은 단일 에이전트 대비 90% 이상의   │
│ 성능 향상을 보였으며, 컨텍스트 엔지니어링이 성능 차이의 80%를 설명하는 핵심 요인으로 분석되었습니다[1].         │
│                                                                                                                 │
│ 구체적 방법론으로는 ① 각 에이전트의 역할과 책임을 명확히 정의하고, ② 시스템 전체의 결정 및 가정(예: 분석 범위,  │
│ 데이터 기준)을 공유 메모리나 프롬프트에 명시적으로 삽입하며, ③ 이전 단계의 핵심 결정만을 요약·전달하여 정보     │
│ 과부하를 방지합니다. 또한, 에이전트 간 충돌 방지를 위해 일관된 포맷, 스타일, 인용 규칙 등을 컨텍스트에          │
│ 포함시키고, 중복 작업이나 모순된 결과가 발생하지 않도록 동적 메모리 참조 및 역할 기반 정보 격리를               │
│ 적용합니다[1][4][5].                                                                                            │
│                                                                                                                 │
│ ## 3. 파일시스템 기반 컨텍스트 관리 시스템에서의 Context Engineering 적용 사례와 특징                           │
│                                                                                                                 │
│ 파일시스템 기반 컨텍스트 관리 시스템은 에이전트가 외부 메모리(파일, DB, shared memory 등)에 정보를              │
│ 저장·조회함으로써, 컨텍스트 윈도우 한계를 극복하고 장기적 일관성을 유지하는 데 중점을 둡니다. 예를 들어,        │
│ 멀티에이전트 시스템에서 각 에이전트가 자신의 결정, 중간 산출물, 외부 데이터 등을 파일시스템에 기록하고, 필요 시 │
│ 유사도 검색(RAG)이나 메타데이터 필터링을 통해 관련 정보를 동적으로 불러와 컨텍스트로 활용합니다[4].             │
│                                                                                                                 │
│ 이러한 구조는 ① 정보의 영속성(장기 기억), ② 역할별/에이전트별 컨텍스트 격리, ③ 동적 정보 선택 및 압축, ④ 시스템 │
│ 전체의 투명성(감사·재현성) 등에서 강점을 가집니다. 실제로 Vellum, LangChain 등 최신 플랫폼은 

## 수행 한 LangSmith 결과물

> https://smith.langchain.com/public/68a0a3a8-d6b3-4c07-a8a3-445d8c6cf663/r